# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 12:42PM,242447,10,8304684,Yusen – 3D Matrix,Released
1,Jul 21 2022 12:38PM,242446,12,HH6446,Hush Hush,Released
2,Jul 21 2022 12:30PM,242445,12,8304675,"SD Head USA, LLC",Released
3,Jul 21 2022 12:26PM,242444,10,8304113,Beach Products Inc,Released
4,Jul 21 2022 12:26PM,242444,10,8304114,Beach Products Inc,Released
5,Jul 21 2022 12:26PM,242444,10,8304115,Beach Products Inc,Released
6,Jul 21 2022 12:26PM,242444,10,8304116,Beach Products Inc,Released
7,Jul 21 2022 12:26PM,242444,10,8304117,Beach Products Inc,Released
8,Jul 21 2022 12:26PM,242444,10,8304118,Beach Products Inc,Released
9,Jul 21 2022 12:26PM,242444,10,8304119,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,242443,Released,4
37,242444,Released,21
38,242445,Released,1
39,242446,Released,1
40,242447,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242443,NaN,NaN,4.0
242444,NaN,NaN,21.0
242445,NaN,NaN,1.0
242446,NaN,NaN,1.0
242447,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242340,0.0,0.0,1.0
242348,0.0,34.0,44.0
242365,16.0,6.0,0.0
242375,0.0,0.0,1.0
242376,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242340,0,0,1
242348,0,34,44
242365,16,6,0
242375,0,0,1
242376,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242340,0,0,1
1,242348,0,34,44
2,242365,16,6,0
3,242375,0,0,1
4,242376,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242340,,,1
1,242348,,34,44
2,242365,16,6,
3,242375,,,1
4,242376,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix
1,Jul 21 2022 12:38PM,242446,12,Hush Hush
2,Jul 21 2022 12:30PM,242445,12,"SD Head USA, LLC"
3,Jul 21 2022 12:26PM,242444,10,Beach Products Inc
24,Jul 21 2022 12:22PM,242443,10,Emerginnova
28,Jul 21 2022 12:02PM,242442,12,"SD Head USA, LLC"
29,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.
42,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.
55,Jul 21 2022 11:30AM,242434,10,Bio-PRF
56,Jul 21 2022 11:22AM,242433,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,,,1
1,Jul 21 2022 12:38PM,242446,12,Hush Hush,,,1
2,Jul 21 2022 12:30PM,242445,12,"SD Head USA, LLC",,,1
3,Jul 21 2022 12:26PM,242444,10,Beach Products Inc,,,21
4,Jul 21 2022 12:22PM,242443,10,Emerginnova,,,4
5,Jul 21 2022 12:02PM,242442,12,"SD Head USA, LLC",,1,
6,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,,,13
7,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,,,13
8,Jul 21 2022 11:30AM,242434,10,Bio-PRF,,,1
9,Jul 21 2022 11:22AM,242433,10,Bio-PRF,,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,1,,
1,Jul 21 2022 12:38PM,242446,12,Hush Hush,1,,
2,Jul 21 2022 12:30PM,242445,12,"SD Head USA, LLC",1,,
3,Jul 21 2022 12:26PM,242444,10,Beach Products Inc,21,,
4,Jul 21 2022 12:22PM,242443,10,Emerginnova,4,,
5,Jul 21 2022 12:02PM,242442,12,"SD Head USA, LLC",,1,
6,Jul 21 2022 11:55AM,242441,10,Eywa Pharma Inc.,13,,
7,Jul 21 2022 11:43AM,242439,20,Alumier Labs Inc.,13,,
8,Jul 21 2022 11:30AM,242434,10,Bio-PRF,1,,
9,Jul 21 2022 11:22AM,242433,10,Bio-PRF,10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,1,,
1,Jul 21 2022 12:38PM,242446,12,Hush Hush,1,,
2,Jul 21 2022 12:30PM,242445,12,"SD Head USA, LLC",1,,
3,Jul 21 2022 12:26PM,242444,10,Beach Products Inc,21,,
4,Jul 21 2022 12:22PM,242443,10,Emerginnova,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Jul 21 2022 12:38PM,242446,12,Hush Hush,1.0,NaN,NaN
2,Jul 21 2022 12:30PM,242445,12,"SD Head USA, LLC",1.0,NaN,NaN
3,Jul 21 2022 12:26PM,242444,10,Beach Products Inc,21.0,NaN,NaN
4,Jul 21 2022 12:22PM,242443,10,Emerginnova,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Jul 21 2022 12:38PM,242446,12,Hush Hush,1.0,0.0,0.0
2,Jul 21 2022 12:30PM,242445,12,"SD Head USA, LLC",1.0,0.0,0.0
3,Jul 21 2022 12:26PM,242444,10,Beach Products Inc,21.0,0.0,0.0
4,Jul 21 2022 12:22PM,242443,10,Emerginnova,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2908984,99.0,31.0,16.0
12,727333,2.0,1.0,0.0
15,727271,6.0,13.0,1.0
16,1212032,8.0,0.0,0.0
19,727225,7.0,9.0,0.0
20,484787,57.0,34.0,0.0
21,1696704,7.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2908984,99.0,31.0,16.0
1,12,727333,2.0,1.0,0.0
2,15,727271,6.0,13.0,1.0
3,16,1212032,8.0,0.0,0.0
4,19,727225,7.0,9.0,0.0
5,20,484787,57.0,34.0,0.0
6,21,1696704,7.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,99.0,31.0,16.0
1,12,2.0,1.0,0.0
2,15,6.0,13.0,1.0
3,16,8.0,0.0,0.0
4,19,7.0,9.0,0.0
5,20,57.0,34.0,0.0
6,21,7.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,99.0
1,12,Released,2.0
2,15,Released,6.0
3,16,Released,8.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,16.0,0.0,1.0,0.0,0.0,0.0,0.0
Executing,31.0,1.0,13.0,0.0,9.0,34.0,0.0
Released,99.0,2.0,6.0,8.0,7.0,57.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,16.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,31.0,1.0,13.0,0.0,9.0,34.0,0.0
2,Released,99.0,2.0,6.0,8.0,7.0,57.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,16.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,31.0,1.0,13.0,0.0,9.0,34.0,0.0
2,Released,99.0,2.0,6.0,8.0,7.0,57.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()